In [19]:
from ngsolve import *
from netgen.geom2d import SplineGeometry
from ngsolve.webgui import Draw
from netgen.occ import *
import numpy as np

T = 0.5 # length of the time interval
shape = Rectangle(1,T).Face()
shape.edges.Min(X).name="left"
shape.edges.Max(X).name="right"
shape.edges.Min(Y).name="bottom"
shape.edges.Max(Y).name="top"
mesh = Mesh(OCCGeometry(shape, dim=2).GenerateMesh(maxh=0.1))
Draw(mesh)

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

In [20]:
order = 8
fes = H1(mesh, order=order, dirichlet="bottom|right|left")
u,v = fes.TnT()
n = specialcf.normal(2)
w = 1 # the wind speed
eps = 1 # small parameter for the diffusion term

h = specialcf.mesh_size
jac = specialcf.JacobianMatrix(mesh.dim)
M = CF( [[2/sqrt(3), 1/sqrt(3)], [1/sqrt(3), 2/sqrt(3)]] )
print(M.dim)
expr = jac.trans * M * jac
print(type(expr))

Cinv = 36 # depends on order in general
diffusion = 1

w0 = GridFunction(L2(mesh,order=0))
w0.Set(1)
ws = CF((w,1))
tau =  InnerProduct(ws,expr *ws)

gamma_0 = 1
#gamma_T = gamma_0 * h**2 / (w * order**2)
#gamma_T = gamma_0 * h / sqrt(w**2 + Cinv * diffusion)
gamma_T = 1/sqrt(tau + (Cinv*eps/h**2)**2)

def gradx(u):
    return grad(u)[0]
def dt(u):
    return grad(u)[1]

uxx = u.Operator("hesse")[0,0]

a = BilinearForm(fes, symmetric=False)
a += eps*gradx(u)*gradx(v)*dx + (ws*grad(u))*v*dx
a += gamma_T * (-eps*uxx + ws * grad(u))*(ws* grad(v)) * dx
a.Assemble()

f = LinearForm(fes)
f += 0 * v * dx
f.Assemble()

gfu = GridFunction(fes)
gfu.Set(exp(x/(2*eps))*sin(pi*x),BND)  # initial condition
Draw(gfu)

res = f.vec.CreateVector()
res.data = f.vec - a.mat * gfu.vec
gfu.vec.data += a.mat.Inverse(fes.FreeDofs()) * res
Draw(gfu)

l2error = sqrt(Integrate((gfu-sin(pi*x)*exp(x/(2*eps))*exp(-y*(eps*pi**3+1/(4*eps))))**2, mesh))
print(l2error)
Draw(sin(pi*x)*exp(x/(2*eps))*exp(-y*(eps*pi**3+1/(4*eps))), mesh)

1
<class 'ngsolve.fem.CoefficientFunction'>


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

0.1195779842178611


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene